# Tarea 2. NN with Linear regression & perceptron, Problema 2

## Fundamentos de inteligencia computacional
## por: Juan Pablo Arango A. & Simón Zapata Caro

In [ ]:
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
from pydoc import help  # can type in the python console `help(name of function)` to get the documentation
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import scale
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from scipy import stats
from IPython.display import display, HTML
from pandas.tools.plotting import scatter_matrix
from matplotlib.artist import setp
import time

plt.rcParams['figure.figsize'] = (10,16)

In [ ]:
df1     = pd.io.excel.read_excel("DatosPunto2.xlsx", sheetname=0)
letters = df1.iloc[:,0].copy()
df1 = df1.iloc[:,1:16].copy()
#normalización min-max
str_comp = 'ARNGOZPT'
df1 = (df1-np.min(df1, axis = 0))/(np.max(df1, axis = 0)-np.min(df1, axis = 0))
#df1 = (df1-np.mean(df1, axis = 0))/(np.std(df1, axis = 0))
X = np.array(df1)

y = -np.ones((X.shape[0],))
i = 0
for L in letters:
    if str_comp.find(L) >= 0:
        print('letra '+str(L)+' aparece en apellidos')
        y[i] = 1
    else:
        print('letra '+str(L)+' no aparece en apellidos')
    i += 1

In [ ]:
X_original = X

Nsamples = X.shape[0]
Ntrain   = int(0.7*Nsamples)
Nval     = int(0.15*Nsamples)
Ntest    = Nsamples-Ntrain-Nval

X_test = X[(Nsamples-Ntest):Nsamples,:]
y_test = y[(Nsamples-Ntest):Nsamples  ]

Xv = X[0:(Nsamples-Ntest),:]
yv = y[0:(Nsamples-Ntest)  ]

In [ ]:
def derivating(fx):
    difcoefs = [1,-1]
    return np.convolve(fx,difcoefs,mode='same')

def chunkIt(seq, num):
    avg = len(seq) / float(num)
    out = []
    last = 0.0

    while last < len(seq):
        out.append(seq[int(last):int(last + avg)])
        last += avg

    return out

def clearList(L): #arroja una lista de L elementos vacios, no sabia de qué otra forma hacerla
    listica = []
    for i in range(0,L):
        listica.append([]);
    return listica

def sigmoid(x):
    return 1/(1+np.exp(-x))

def Perceptron_training(X,y,iteraciones=500,de_min=0.01):
    w_array     = []
    p_i         = []
    
    w_          = np.zeros(X.shape[1] + 1,)
    n           = X.shape[0]
    nf          = X.shape[1]
    sw          = 0
    j           = 0
    index       = []
    error_array = []
    
    while j < iteraciones and sw == 0:
        print('iteración: '+str(j+1))
        for i in range(0,n):
            g1  = np.sum(w_*np.insert(X[i,:],0,1))
            yp  = 2*sigmoid(g1)-1
            if yp*y[i] <= 0:
                w_ = w_ + np.insert(X[i,:],0,1)*y[i]
        
        w_array.append(w_)
        predicted = []
        for i in range(0,n):
            predicted.append(Predict(X[i,:],w_))
        p_i.append(predicted)
        E = 0
        for i in range(0,n):
            if predicted[i]*y[i]<0:
                E += -predicted[i]*y[i]
        index.append(j)
        error_array.append(E)
        #print(d_error_array[j])
        
        #if np.abs(d_error_array[len(d_error_array)-1]) < de_min:
        #   sw = 1
        
        j = j + 1
    #print('Función de error: '+str(E[0])+' iterando ' +str(j)+' veces')
    plt.plot(error_array,'b')
    plt.title('función de error')
    plt.xlabel('Iteraciones')
    plt.show()
    
    iter_min = np.argmin(error_array)
    print('iteracción óptima: '+str(iter_min)+ ', función de error: '+str(error_array[iter_min]))
    return [w_array[iter_min],p_i[iter_min]]

def Predict(X,w_):
    g = np.dot(w_.T,np.insert(X,0,1))
    return 2*sigmoid(g)-1

### Entrenamiento de la red neuronal usando validación bootstraping 

In [ ]:
from sklearn.model_selection import KFold
X = Xv
y = yv
Nf = 10
N  = X.shape[0]
kf = KFold(n_splits=Nf)

error_train = np.zeros((Nf,))
error_val   = np.zeros((Nf,))
k=0
error_min = np.inf
for train_index, test_index in kf.split(X):
    wPerceptron = []
    predicted1  = []
    predicted2  = []
    error1      = []
    error2      = []
    [wPerceptron,predicted1] = Perceptron_training(X[train_index],y[train_index],de_min=0.001)
    ecm1[i] = np.sqrt(np.sum(((predicted1[i]-y[train_index])*(predicted1[i]-y[train_index])))/y[train_index].shape)[0]

    p_i = []
    for j in range(0,X[test_index].shape[0]):
        p_i.append(Predict(X[test_index][j,:],wLMS[i]))
    predicted2[i] = p_i
    ecm2[i] = np.sqrt(np.sum(((predicted2[i]-y[test_index])*(predicted2[i]-y[test_index])))/y[test_index].shape)[0]

    if(ecm2[i] < minE_val):
        wLMS_optimo = wLMS[i]
        minE_val = ecm2[i]
        alphaOptimo = alpha[i]
    print('validación ',(k+1),' Función de error: ',str(E),' iterando ',str(Ni),' veces, con alpha: ',str(alpha[i]))
ecm_train[k,:] = ecm1
ecm_val[k,:]   = ecm2
k+=1

In [ ]:
[wPERC,predicted1] = Perceptron_training(X_train,y_train,iteraciones=50,de_min=0.001)
Accuracy_train = np.sum(np.array(predicted1)*y_train > 0)/y_train.shape[0]
predicted2 = []
for j in range(0,X_val.shape[0]):
    predicted2.append(Predict(X_val[j,:],wPERC))
Accuracy_val = np.sum(np.array(predicted2)*y_val > 0)/y_val.shape[0]
d = {'valores predichos': predicted2, 'etiquetas de validación' : y_val, 'multiplicación' : np.array(predicted2)*y_val}
df = pd.DataFrame(data=d)
df


In [ ]:
print('tasa de entrenamiento: '+ str(Accuracy_train)+', tasa de validación: '+str(Accuracy_val))